<a href="https://colab.research.google.com/github/csabi0312/crypto-svm/blob/gru/crypto14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install ujson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 7.1 MB/s eta 0:00:00


In [3]:
pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install oandapyV20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=70035 sha256=01a2db7ba3439fbd6cf73017b9b45a7de75953fdb2461e841162f56aedd6344b
  Stored in directory: /root/.cache/pip/wheels/d2/b6/8e/6fe7d26410a99ddb24ac4358691c5feabf4320d20d8053ca25
Successfully built oandapyV20


In [5]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.7 MB/s eta 0:00:00


In [6]:
pip install v20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 KB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for v20: filename=v20-3.0.25.0-py3-none-any.whl size=85780 sha256=9d6397c42eb0d7a6d106aaf38020790392a890be6ab626d7fceacc257af75b4b
  Stored in directory: /root/.cache/pip/wheels/9b/50/e6/7431cad84aed36618f1cd68ee913ba2d8b7759e3c4aecfe2d5
Successfully built v20


In [7]:
pip install --upgrade git+https://github.com/yhilpisch/tpqoa.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/yhilpisch/tpqoa.git to /tmp/pip-req-build-bgvidjvu
  Running command git clone --filter=blob:none --quiet https://github.com/yhilpisch/tpqoa.git /tmp/pip-req-build-bgvidjvu
  Resolved https://github.com/yhilpisch/tpqoa.git to commit f47a1ffddd2b2684c7e1569df6210b98dbe4e701
  Preparing metadata (setup.py) ... done
  Created wheel for tpqoa: filename=tpqoa-0.0.55-py3-none-any.whl size=10928 sha256=607de37e49370b57afff4ad6b2603206bd56e0a4fae3834a27941866d0fa8ab1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ti7azgba/wheels/54/90/ca/b84ec93e1d9679237c36be01b355ed94b670276656689e84a5
Successfully built tpqoa


In [8]:
import pickle
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from numpy import trapz
import pickle
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from numpy import trapz
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import keras.losses
import sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras_tuner
from tensorflow.keras import regularizers
import tpqoa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import load_model
import oandapyV20.endpoints.instruments as instruments
import datetime
import oandapyV20.endpoints.instruments as instruments
import datetime
from sklearn.metrics import mean_squared_error as mse

In [9]:
pip install rich

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.4 MB/s eta 0:00:00
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1


In [10]:
from rich.progress import track

# DATA

In [18]:
raw = pd.read_csv("/content/drive/MyDrive/Crypto/new3/full_spread_data.csv", parse_dates = ["time"], index_col = "time").dropna()
raw["returns"] =np.log(raw.Close / raw.Close.shift(1))
raw.drop(["Open","Spread"],axis=1,inplace=True)
cols = []
for lag in range(1, 15):
    col_h = "h_lag{}".format(lag)
    col_l = "l_lag{}".format(lag)
    col_r = "c_lag{}".format(lag)
    col_v = "v_lag{}".format(lag)
    raw[col_h] = raw.High.shift(lag)
    cols.append(col_h)
    raw[col_l] = raw.Low.shift(lag)
    cols.append(col_l)
    raw[col_r] = raw.Close.shift(lag)
    cols.append(col_r)
    raw[col_v] = raw.Volume.shift(lag)
    cols.append(col_v)
    raw.dropna(inplace = True)
raw.drop(["High","Low","Volume","Close"],axis=1,inplace=True)

raw["returns2"]=(raw["returns"]>=0).astype(int)

to=int(len(raw)*0.8)
X_train=raw.drop(["returns2","returns"],axis=1).iloc[:to].to_numpy()
y_train=raw["returns2"].iloc[:to].to_numpy().astype(float)
X_test=raw.drop(["returns2","returns"],axis=1).iloc[to:].to_numpy()
y_test=raw["returns2"].iloc[to:].to_numpy().astype(float)

scaler1 = MinMaxScaler()
X_train=scaler1.fit_transform(X_train)
X_test=scaler1.transform(X_test)

from tensorflow.keras.utils import to_categorical
y_train2 = to_categorical(y_train, 2)
y_test2 = to_categorical(y_test, 2)

print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(1595, 56) (1595,) (399, 56) (399,)


In [30]:
raw = pd.read_csv("/content/drive/MyDrive/Crypto/new3/full_spread_data.csv", parse_dates = ["time"], index_col = "time").dropna()
raw["returns"] =np.log(raw.Close / raw.Close.shift(1))
raw["returns2"]=(raw["returns"]>=0).astype(int)
raw.drop(["Open","Spread","Close","High","returns","Volume","Low"],axis=1,inplace=True)
cols = []
for lag in range(1, 30):
    col_r = "r_lag{}".format(lag)
    raw[col_r] = raw.returns2.shift(lag)
    raw.dropna(inplace = True)

to=int(len(raw)*0.8)
X_train=raw.drop(["returns2"],axis=1).iloc[:to].to_numpy()
y_train=raw["returns2"].iloc[:to].to_numpy().astype(int)
X_test=raw.drop(["returns2"],axis=1).iloc[to:].to_numpy()
y_test=raw["returns2"].iloc[to:].to_numpy().astype(int)

from tensorflow.keras.utils import to_categorical
y_train2 = to_categorical(y_train, 2)
y_test2 = to_categorical(y_test, 2)

print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(1331, 29) (1331,) (333, 29) (333,)


In [31]:
raw

,returns2,r_lag1,r_lag2,r_lag3,r_lag4,r_lag5,r_lag6,r_lag7,r_lag8,r_lag9,...,r_lag20,r_lag21,r_lag22,r_lag23,r_lag24,r_lag25,r_lag26,r_lag27,r_lag28,r_lag29
time,,,,,,,,,,,,,,,,,,,,,
2017-03-14 21:00:00+00:00,0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
2017-03-15 21:00:00+00:00,0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
2017-03-16 21:00:00+00:00,0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
2017-03-17 21:00:00+00:00,1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
2017-03-18 21:00:00+00:00,0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-17 22:00:00+00:00,1,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
2023-01-18 22:00:00+00:00,1,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
2023-01-19 22:00:00+00:00,1,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0


# GRU2

In [33]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units1', min_value=5, max_value=200, step=1),activation="relu",input_shape=(29,1)))
    model.add(Dense(hp.Int('units2', min_value=5, max_value=180, step=1),activation="relu"))
    model.add(Dense(hp.Int('units3', min_value=5, max_value=160, step=1),activation="relu"))
    model.add(Dense(hp.Int('units4', min_value=5, max_value=140, step=1),activation="relu"))
    model.add(Dense(hp.Int('units5', min_value=5, max_value=120, step=1),activation="relu"))
    model.add(GRU(hp.Int('units6', min_value=1, max_value=120, step=1), return_sequences=True))
    model.add(LSTM(hp.Int('units7', min_value=1, max_value=120, step=1), return_sequences=False))
    model.add(Dense(2,activation="softmax"))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

tuner = keras_tuner.RandomSearch(
      build_model,
      objective='val_loss',
      max_trials=1,overwrite = True)
tuner.search(X_train,y_train2 , epochs=10, validation_split=0.2)
best_model = tuner.get_best_models()[0]
best_model.summary()

early_stopping=EarlyStopping(patience=25, verbose=0)
checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=1)
best_model.fit(X_train,y_train2, batch_size=50,epochs=100, verbose=0, \
                validation_split=0.2,callbacks=[checkpointer, early_stopping], shuffle=True)

predictions =best_model.predict(X_test)
pred_sign=np.argmax(predictions,axis=1)
true_sign=np.argmax(y_test2,axis=1)
print(confusion_matrix(true_sign,pred_sign))
print(classification_report(true_sign,pred_sign))

Trial 1 Complete [00h 00m 14s]
val_loss: 0.6836357712745667

Best val_loss So Far: 0.6836357712745667
Total elapsed time: 00h 00m 14s
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 29, 68)            136       
                                                                 
 dense_1 (Dense)             (None, 29, 126)           8694      
                                                                 
 dense_2 (Dense)             (None, 29, 141)           17907     
                                                                 
 dense_3 (Dense)             (None, 29, 70)            9940      
                                                                 
 dense_4 (Dense)             (None, 29, 86)            6106      
                                                                 
 gru (GRU)                   (None, 29, 40)           

In [ ]:
pred_sign=-(predictions>=0).astype(int)+(predictions<0).astype(int)
true_sign=(y_true>=0).astype(int)-(y_true<0).astype(int)
print(confusion_matrix(true_sign,pred_sign))
print(classification_report(true_sign,pred_sign))

[[215  75]
 [216  92]]
              precision    recall  f1-score   support

          -1       0.50      0.74      0.60       290
           1       0.55      0.30      0.39       308

    accuracy                           0.51       598
   macro avg       0.52      0.52      0.49       598
weighted avg       0.53      0.51      0.49       598



In [ ]:
X_test.shape

(598, 56)

# GRU

In [ ]:
def build_model(hp):
    model = Sequential()

    model.add(LSTM(hp.Int('units2', min_value=1, max_value=200, step=1), return_sequences=True, input_shape=(60,1)))
    model.add(GRU(hp.Int('units2', min_value=1, max_value=200, step=1), return_sequences=True))
    model.add(LSTM(hp.Int('units2', min_value=1, max_value=200, step=1), return_sequences=False))
    model.add(Dense(hp.Int('units3', min_value=5, max_value=100, step=1)))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

In [ ]:
tuner = keras_tuner.RandomSearch(
      build_model,
      objective='val_loss',
      max_trials=10,overwrite = True)
tuner.search(X_train,y_train , epochs=10, validation_split=0.2)
best_model = tuner.get_best_models()[0]
best_model.summary()

Trial 10 Complete [00h 00m 11s]
val_loss: 0.8127514123916626

Best val_loss So Far: 0.8071630597114563
Total elapsed time: 00h 02m 12s
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 109)           48396     
                                                                 
 gru (GRU)                   (None, 60, 109)           71940     
                                                                 
 lstm_1 (LSTM)               (None, 109)               95484     
                                                                 
 dense (Dense)               (None, 29)                3190      
                                                                 
 dense_1 (Dense)             (None, 1)                 30        
                                                                 
Total params: 219,040
Trainable params: 219,040
Non-t

In [ ]:
#best_model=build_best()
early_stopping=EarlyStopping(patience=100, verbose=0)
checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=1)
best_model.fit(X_train,y_train, batch_size=32,epochs=1000, verbose=0, \
                validation_split=0.1,callbacks=[checkpointer, early_stopping], shuffle=True)


Epoch 1: val_loss improved from inf to 1.13291, saving model to weights1.hdf5

Epoch 2: val_loss did not improve from 1.13291

Epoch 3: val_loss did not improve from 1.13291

Epoch 4: val_loss improved from 1.13291 to 1.13181, saving model to weights1.hdf5

Epoch 5: val_loss did not improve from 1.13181

Epoch 6: val_loss did not improve from 1.13181

Epoch 7: val_loss did not improve from 1.13181

Epoch 8: val_loss did not improve from 1.13181

Epoch 9: val_loss did not improve from 1.13181

Epoch 10: val_loss did not improve from 1.13181

Epoch 11: val_loss did not improve from 1.13181

Epoch 12: val_loss did not improve from 1.13181

Epoch 13: val_loss did not improve from 1.13181

Epoch 14: val_loss did not improve from 1.13181

Epoch 15: val_loss did not improve from 1.13181

Epoch 16: val_loss did not improve from 1.13181

Epoch 17: val_loss did not improve from 1.13181

Epoch 18: val_loss did not improve from 1.13181

Epoch 19: val_loss did not improve from 1.13181

Epoch 20: v

In [ ]:
predictions = scaler2.inverse_transform(best_model.predict(X_test)).flatten()
y_true=scaler2.inverse_transform(y_test.reshape(-1,1)).flatten()
pred_sign=(predictions>=0).astype(int)-(predictions<0).astype(int)
true_sign=(y_true>=0).astype(int)-(y_true<0).astype(int)
print(confusion_matrix(true_sign,pred_sign))
print(classification_report(true_sign,pred_sign))

2/2 [==============================] - 1s 9ms/step
[[11 12]
 [21 10]]
              precision    recall  f1-score   support

          -1       0.34      0.48      0.40        23
           1       0.45      0.32      0.38        31

    accuracy                           0.39        54
   macro avg       0.40      0.40      0.39        54
weighted avg       0.41      0.39      0.39        54

